# AERONAUTICAL PROJECT
The following project is based on the realization of virtual reality software for the aeronautical sector with the purpose of improving the learning experience for flight assistants, the goal is to include artificial intelligence and implement with the metahumans who will be inserted into the software.
for start the project we will begin making a review of some contents.
## Interaction with virtual characters
One of the main ideas of this software is the "interaction with virtual characters" or "communication with virtual entities." In this scenario, "metahumanos" could be virtual characters, avatars or digital entities that can interact and communicate with users in a similar way to how humans interact with each other.
So to develop the idea we need tools for the creation of chatbots and virtual agents with advanced natural language processing capabilities.

In [ ]:
!pip install nltk

In [ ]:
!pip install tensorflow

In [ ]:
!pip install requests

## First thing we do is import all the libraries
we need and download nltk.
1. nltk (NATURAL LENGUAGE TOOLKIT) : used for natural language processing.
2. Wordnetlemmatizer: A lematizer is used to reduce words to its base or motto, which can be useful in text processing.
3. Lemmatizer = Wordnetlemmatizer (): Create an instance of the lematizer
4. Json: Our archive is made with Json.
5. Pickle: Used for saving trained models
6. Keras: for the Neural network.

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
import requests

Let's  3 create empty lists.
1. Words: stores the words we will process.
2. Classes: will store the classes.
3. Documments: will be used to save text documents.

In [ ]:
words=[]
classes = []
documents = []

In [ ]:
raw_url = "https://raw.githubusercontent.com/NataliaGutman/AERONAUTICAL-PROJECT/f4261aff6dcf8e41164f57232c1abacd783792d8/intents.json"

response = requests.get(raw_url)

if response.status_code == 200:
    datos_json = json.loads(response.text)
else:
    print("Failed to retrieve data from GitHub.")

We open our Json archive.

let's check if it is working correctly

In [ ]:
print(datos_json) #it is working fine but we need to keep cleanning the data

{'intents': [{'tag': 'Greetings', 'text': ['Bienvenida a bordo', 'Disculpe', 'Gracias por volar con nosotros'], 'response': ['Disculpe, podria ayudarme a abrochar mi cinturon de seguridad, por favor', 'Perdon por molestar, podria brindarme asistencia para abrochar mi cinturon de seguridad, por favor', 'Necesito un poco de ayuda con mi cinturon, puede asistirme', 'Sería tan amable de abrochar mi cinturon de seguridad', 'Necesito ayuda con el cinturon de seguridad, podria, por favor', 'Podria ofrecerme asistencia con el cinturon', 'Disculpe, puede ocuparse de mi cinturon, por favor', 'Por favor, me podria dar una mano con el cinturon', '¿Seria tan amable de abrochar mi cinturon de seguridad', 'Necesito asistencia para abrochar mi cinturon, podria', 'Podría brindarme ayuda con el cinturon de seguridad, por favor', 'Disculpe, me podría dar una mano para abrochar el cinturon', '¿Podria ocuparse de mi cinturon de seguridad, por favor', 'Por favor, puede ayudarme a abrochar mi cinturon de seg

In [ ]:
import nltk
nltk.download('punkt')

for intent in datos_json['intents']:
    for text in intent['text']:
        w = nltk.word_tokenize(text)
        words.extend(w)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We open the document list and add the tokenization, intent is the label if the name of the intent is not on the class list, it is added to this list. This ensures that Classses will contain all the unique labels of the intentions present in the data.

In [ ]:
documents.append((w, intent['tag']))
if intent['tag'] not in classes:
    classes.append(intent['tag'])

We lemmatize W.lower is to put them in lowercase, Sorted is to hate alphabically for not repeat the same word.

In [ ]:
nltk.download('wordnet')
words = [lemmatizer.lemmatize(w.lower())
for w in words]
words = sorted(list(set(words)))

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
classes = sorted(list(set(classes)))

In [ ]:
print(len(documents), "documents")

1 documents


In [ ]:
print(len(classes), "classes", classes)

1 classes ['welcome']


We print the lemmatized words also we safe with pickles the information of the words and the classes

In [ ]:
print(len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

30 unique lemmatized words ['a', 'ayudarle', 'bienvenida', 'bordo', 'buen', 'buenas', 'buenos', 'claro', 'con', 'de', 'deme', 'dia', 'disculpe', 'gracias', 'inmediato', 'momento', 'noches', 'nosotros', 'permitame', 'por', 'puedo', 'segundo', 'si', 'su', 'supuesto', 'tardes', 'tiquete', 'un', 'ver', 'volar']


In [ ]:
training = []

In [ ]:
output_empty = [0] * len(classes)

In [ ]:
for doc in documents:
    bag = []
pattern_words = doc[0]

In [ ]:
 pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

We create a list binary list

In [ ]:
bag.append(1) if w in pattern_words else bag.append(0)

This code is used to prepare training data for a text classification model

In [ ]:
output_row = list(output_empty)
output_row[classes.index(doc[1])] = 1
training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training = np.array(training)

In [ ]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


We create the Neural network

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [ ]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 1s 930ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 22ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 21ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accurac

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


To predict the sentences and get a response from the user that allows us to create a new file “chatapp.py”.
We will load the trained model and then use a graphical user interface that will predict the bot's response. The model will only tell us the class it belongs to, so we will implement some functions that will identify the class and then retrieve a random answer from the list of answers.
Again we import the necessary packages and load the pickle files “words.pkl” and “classes.pkl” that we have created when we trained our model

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random

In [ ]:
raw_url = "https://raw.githubusercontent.com/NataliaGutman/AERONAUTICAL-PROJECT/f4261aff6dcf8e41164f57232c1abacd783792d8/intents.json"

response = requests.get(raw_url)

if response.status_code == 200:
    intents = json.loads(response.text)
    print("Archive found")
else:
    print("Failed to retrieve data from GitHub for intents.json")

Archive found


In [ ]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [1]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
    # return bag of words array: 0 or 1 for each word in the bag that

In [15]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)


In [33]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []

    # Resto de tu código aquí

    return return_list
    for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
      return return_list

In [34]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res